# ASCL - Astrophysics Source Code Library

"Software is the most used instrument in astronomy"

-- Alice Allen, ASCL

In [2]:
# import necessary libs
import requests 
import urllib.request, json 

In [4]:
with urllib.request.urlopen("http://ascl.net/code/json") as url:
    ascl_data = json.loads(url.read().decode())
    #print(data)

# +++ for testing, just grab the first 3
ascl_data = {k: ascl_data[k] for k in ('1', '2', '3')}

ascl_data

{'1': {'ascl_id': '9903.001',
  'title': 'LENSKY: Galactic Microlensing Probability',
  'credit': 'Nemiroff, Robert J.',
  'abstract': 'Given a model for the Galaxy, this program computes the microlensing rate in any direction. Program features include the ability to include the brightness of the lens and to compute the probability of lens detection at any level of lensing amplification. The program limits itself to lensing by single stars of single sources. The program is currently setup to accept input from the Galactic models of Bahcall and Soniera (1982, 1986).\n\nThere are three files needed for LENSKY, the Fortran file lensky.for and two input files: galmod.dsk (15 Megs) and galmod.sph (22 Megs). The zip file available below contains all three files. The program generates output to the file lensky.out. The program is pretty self-explanatory past that.',
  'topic_id': '20392',
  'bibcode': '1999ascl.soft03001N',
  'views': '2033',
  'preferred_citation': '',
  'site_list': ['http:

In [19]:
list_refs = ascl_data['1']['ref_list']
list_refs


for ref in list_refs:
  print(list_refs[0]) 

http://adsabs.harvard.edu/abs/1998ApJ...509...39N


In [10]:
# create a dict data structure to hold the keywords
ascl_keywords_dict = {}

In [14]:
ascl_keywords_dict[ascl_data['1']['ascl_id']] = ascl_data['1']['ref_list']
ascl_keywords_dict

{'9903.001': ['http://adsabs.harvard.edu/abs/1998ApJ...509...39N']}

In [34]:
ref_url = ascl_data['1']['ref_list'][0]
ref_page = requests.get(ref_url)
if ref_page.status_code != 200:
    print("URL did not work for ASCL ID", ascl_data['1']['ascl_id'], ref_url)

In [31]:
ascl_data['1']['ref_list'][0]

'http://adsabs.harvard.edu/abs/1998ApJ...509...39N'